In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from transformers import AutoTokenizer, AutoModel, DataCollatorWithPadding
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [3]:
MODEL = "bert-base-uncased"
BATCH_SIZE = 16
NUM_EPOCHS = 1
LEARNING_RATE = 1e-5
RANDOM_SEED = 12345
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

In [22]:
df = pd.read_pickle("train/seq2reply_regression_data.pickle")
df = df.loc[df.clean_text.notnull()]
NUM_CLASSES = max(df.num_replies.unique()) + 1
print(f"num_classes: {NUM_CLASSES}")

num_classes: 345


In [5]:
class OrdinalUfoDataset(torch.utils.data.Dataset):
    def __init__(self, X, y, tokenizer, num_classes):
        self.encodings = tokenizer(X.tolist(), truncation=True)
        self.labels = y.tolist()
        self.num_classes = num_classes

    def __getitem__(self, index):
        item = {key: torch.tensor(val[index]) for key, val in self.encodings.items()}
        label = self.labels[index]
        item["labels"] = torch.tensor(label)
        levels = [1] * label + [0] * (self.num_classes - 1 - label)
        item["levels"] = torch.tensor(levels, dtype=torch.float32)
        return item

    def __len__(self):
        return len(self.labels)

In [6]:
# split the data
X_train, X_test, y_train, y_test = train_test_split(
    df.clean_text,
    df.num_replies.astype("int16"),
    test_size=0.2,
    random_state=RANDOM_SEED,
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=RANDOM_SEED
)

# initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL)

# initialize datasets
train_dataset = OrdinalUfoDataset(X_train, y_train, tokenizer, NUM_CLASSES)
val_dataset = OrdinalUfoDataset(X_val, y_val, tokenizer, NUM_CLASSES)
test_dataset = OrdinalUfoDataset(X_test, y_test, tokenizer, NUM_CLASSES)

# initialize data collator
data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer, padding="longest", return_tensors="pt"
)

# initialize dataloaders
train_loader = DataLoader(
    dataset=train_dataset, collate_fn=data_collator, batch_size=BATCH_SIZE, shuffle=True
)
val_loader = DataLoader(
    dataset=val_dataset, collate_fn=data_collator, batch_size=BATCH_SIZE, shuffle=True
)
test_loader = DataLoader(
    dataset=test_dataset, collate_fn=data_collator, batch_size=BATCH_SIZE, shuffle=True
)

In [7]:
class OrdinalRegressionBERT(nn.Module):
    def init(self, bert, num_classes, dropout_rate=0.1):
        super(OrdNet, self).init()
        self.num_classes = num_classes

        self.bert = AutoModel.from_pretrained(bert)
        self.dropout = nn.Dropout(p=dropout_rate)
        self.fc = nn.Linear(768, 1, bias=False)
        self.linear_1_bias = nn.Parameter(torch.zeros(self.num_classes - 1).float())

    def forward(self, input_ids, attention_mask):
        hidden = self.bert(input_ids=input_ids, attention_mask=attention_mask)[0]
        print(f"hidden shape: {hidden.shape}")
        x = hidden[:, 0]
        print(f"x shape: {x.shape}")
        x = self.dropout(x)
        logits = self.fc(x)
        print(f"logits shape: {logits.shape}")
        logits = logits + self.linear_1_bias
        probas = torch.sigmoid(logits)
        return logits, probas

In [8]:
def cost_fn(logits, levels, imp):
    val = (-torch.sum((F.logsigmoid(logits)*levels
                      + (F.logsigmoid(logits) - logits)*(1-levels))*imp,
           dim=1))
    return torch.mean(val)

In [ ]:
def eval_metrics(model, data_loader, device):
    mae, mse, num_examples = 0, 0, 0
    for i, (input_ids, _, attention_mask, labels, levels) in enumerate(data_loader):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        logits, probas = model(input_ids, attention_mask)
        
        predicted_labels = torch.sum(probas > 0.5, dim=1)
        num_examples += labels.size(0)
        
        mae += torch.sum(torch.abs(predicted_labels - labels))
        mse += torch.sum((predicted_labels - targets)**2)
    mae = mae.float() / num_examples
    mse = mse.float() / num_examples
    return mae, mse

In [ ]:
model = OrdinalRegressionBERT(MODEL, NUM_CLASSES)
model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
imp = torch.ones(NUM_CLASSES-1, dtype=torch.float32) # task importance weights

In [21]:
for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, (input_ids, _, attention_mask, labels, levels) in enumerate(
        train_loader
    ):
        input_ids = input_ids.to(DEVICE)
        attention_mask = attention_mask.to(DEVICE)
        labels = labels.to(DEVICE)
        levels = levels.to(DEVICE)

        # forward
        logits, probas = model(input_ids, attention_mask)
        cost = cost_fn(logits, levels, imp)
        optimizer.zero_grad()

        # backward
        cost.backward()

        # update
        optimizer.step()

        # log
        s = "Epoch: %03d/%03d | Batch %04d/%04d | Cost: %.4f" % (
            epoch + 1,
            num_epochs,
            batch_idx,
            len(train_dataset) // BATCH_SIZE,
            cost,
        )
        print(s)

    model.eval()
    with torch.set_grad_enabled(False):
        val_mae, val_mse = compute_mae_and_mse(model, val_loader, device=DEVICE)

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels', 'levels'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels', 'levels'])
